Bikalpa Baniya 

Spring 2022 Research Group

baniya@wisc.edu

**Description of the Code** 

The following code prepares data from NLYS79 Employment, Education and other relevant demographic variables. The final data will be used to create models on job transitions. 

The input for the files are in Raw Data folder in the our 2022 Spring Research Group [google drive](https://drive.google.com/drive/folders/1u9yjW4Cf5zZzB0f2GUL6NU-ShHa_k6rt?usp=sharing). The variables currently collected and processed are from the following NLYS79 categories:


*   Weekly Array: Employment Status, Hours Worked, Dual Jobs

*   Employment Summary Statistics: Hourly Wage, Previous Job Number 

*   Education Summary Statistics: Highest Grade Completed, Year Highest Grade Completed, Year High School Completed 

*   Demograhpic Information: Sample ID, Race, Sex, Year of birh


**Structure of Code**

1.   Loading packages and path
2.   Rename Function: There are thousands of variables so we need to create a function to systematically rename all the variables 
3.   Threshold Remove: This removes any observation that has more than 4 years gap in missing observations 
4.   Unique Job Tracker: Individuals can hold multiple jobs at the same time so a tracker was used to identify total number of lifetime jobs and weeks where multiple jobs were held
5.   Weekly Employment History:  The tracker in 4 is then used create a dataframe with weekly employment History 
6.   Wage Data: Wage information is added to each year
7.   Hours Worked Data: Hours worked data is added to each year
8.   Education: The education data is added to each year 
9.   To weekly: Wage, Hours Worked and Eduation were captured as yearly data in 6,7,8. Here these three information is converted to weekly information
10.  Joining: Joinig all the data cleaned above into one dataframe, filtering only the third week of each month, and converting into long format. 



*Due to the large size of the input files, running the entire code will take about an hour. Instead each intermediary set saves its data into a temp folder and one can simply load this file to run the rest of the code.*

#Loading packages and path

In [ ]:
#Import
import pandas as pd
import numpy as np
import re
import gc
import sys
import math
from datetime import date

In [ ]:
#Only run this if this is the first time 
#pip install cpi

In [ ]:
#Mounting the drive to ensure data can be uploaded from Google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Main Path

path_main = '/content/drive/MyDrive/Alpha_beta_gamma/Coding/Raw Data'
path_main_emp = path_main+'/Employment'
path_main_edu = path_main + '/Education/'
path_weekly_array = path_main_emp+'/Employment History/Weekly Array/'
path_byJob = path_main_emp + '/Employment History/By Jobs/'
path_summary_array = path_main_emp+ '/Summary Measures/'

#Rename Functions

In [ ]:
#main output: 
#rename_emp_hist(df, labels, name)

In [ ]:
######Retriving the year variable
def get_year_hist(x):
    x["got_year"] = -111
    if x.year != "XRND":
      x.got_year = x.year
      return x
    if x.year == "XRND":

      #Identify the index of the year 
      list1 = x.var_title.split()
      index=-99
      if list1[0] == "LABOR" and list1[1]=="FORCE":     #If Employment Status
        index = 0
      elif list1[0] == "HOURS" and list1[1]=="AT":   #If Hours Worked 
        index = 0
      elif list1[0] == "JOB" and list1[1]=="NUMBER":    #If Dual Job
        index = 1
      elif list1[2]=="OF" and list1[3]=="JOB":    #If Job Date
        index = 1
      elif list1[2] == "NO" and list1[4]=="GAP":    #If Within Job gap
        index = 2
      elif list1[2] == "GAP" and list1[4]=="NOT":    #If Between Job gap
        index = 1
      elif list1[0] == "PREVIOUS" and list1[1]=="JOB":    #If Previous Job number
        index = 1


      #Getting the year
      split_prep2 = re.sub("[^0-9]", " ", x.var_title)
      list2 = split_prep2.split()
      x.got_year =  list2[index]  
            
      return x

######Retriving the job# variable
def get_job_hist(x):
  x["got_job"] = -111
  list1 = x.var_title.split()
                      
  if "JOB" in list1:
    job_index = list1.index("JOB")
    if job_index != len(list1)-1: 
      x.got_job = re.sub("[^0-9]", "", list1[job_index+1])
      if x.got_job == "":
        x.got_job = re.sub("[^0-9]", "", list1[job_index+2])
        if x.got_job == "":          ###Added fix for Previous Job Number
          x.got_job = re.sub("[^0-9]", "", list1[job_index+6])
    else: 
      x.got_job= ""
  else:
    x.got_job= ""
  return x


######Retriving the week variable
def get_week_hist(x):
  x["got_week"] = -111
  list1 = x.var_title.split()
  if "WEEK" in list1:
    week_index = list1.index("WEEK")
    if list1[week_index+1].isdigit():
      x.got_week = list1[week_index+1]
    else:
      x.got_week = ""
  else: 
    x.got_week = ""
  return x


######Retriving the gap variable
def get_gap_hist(x):
  x["got_gap"] = -111
  list1 = x.var_title.split()
  if "GAP" in list1:
    gap_index = list1.index("GAP")
    x.got_gap = list1[gap_index+1]
  else:
    x.got_gap= ""
  return x


######Retriving the gap type variable
def get_gap_type_hist(x):
  x["got_gap_type"] = -111
  list1 = x.var_title.split()
  if "GAP" in list1 and "NO" in list1 and "WORK," in list1:
    x.got_gap_type = "WithinJob"
  elif "GAP" in list1 and "NOT" in list1 and "WORKING," in list1: 
    x.got_gap_type = "BetweenJob"
  else: 
    x.got_gap_type = ""
  return x

######Retriving the Start/Stop type for Emp Hist Array
def get_Start_Stop (x):
  x["got_Start_Stop"] = -111
  list1 = x.var_title.split()
  if "START" in list1 and "WEEK" in list1:
    x.got_Start_Stop = "Start"
  elif "STOP" in list1 and "WEEK" in list1: 
    x.got_Start_Stop = "Stop"
  else: 
    x.got_Start_Stop = ""
  return x


######Collecting weeks jobs and gaps 
def rename_collect(x, name):
  new_name=""

  #has job#,gap#,start/stop & within/bewteen  =  Within job start/stop.   name_year1985_Start_WithinJob_gap1_job1
  if x.got_week=="" and x.got_job!="" and x.got_gap!="" and x.got_Start_Stop!="" and x.got_gap_type=="WithinJob": 
    new_name = name+"_year"+ x.got_year + "_"+ x.got_Start_Stop+"_" + x.got_gap_type +"_gap"+ x.got_gap +"_job"+ x.got_job

  #has gap#, start/stop & within/bewteen  =  Between job start/stop.      name_year1985_Start_BetweenJob_gap1
  elif x.got_week=="" and x.got_job=="" and x.got_gap!="" and x.got_Start_Stop!="" and x.got_gap_type=="BetweenJob": 
    new_name = name+"_year"+ x.got_year + "_" +x.got_Start_Stop+"_"+ x.got_gap_type + "_gap"+ x.got_gap

  #has start/stop & Job#  =  Job start/stop Week.                         name_year1985_Start_job1
  elif x.got_week=="" and x.got_job!="" and x.got_gap=="" and x.got_Start_Stop!="" and x.got_gap_type=="": 
    new_name = name+"_year"+ x.got_year + "_"+ x.got_Start_Stop+"_job"+ x.got_job

  #has week = Emp_status among other things.                              name_year1985_week94
  elif x.got_week!="" and x.got_job=="" and x.got_gap=="":                 
    new_name = name+"_year"+ x.got_year +"_week"+ x.got_week

  elif x.got_week=="" and x.got_job!="" and x.got_gap=="":                  #has job, like in wage labels and hours worked labels
    new_name = name+"_year"+ str(x.got_year) +"_job"+ str(int(x.got_job))            #The str(int()) to remove 0 from 01

  elif x.got_week!="" and x.got_job!="" and x.got_gap=="":                  #has week, job# 
    new_name = name+"_year"+ x.got_year +"_week"+ x.got_week +"_job"+ x.got_job

  return new_name

##########################################################
###############Building the rename function###############
##########################################################

def rename_emp_hist(df, labels, name):

  #######Creating the new name 

  labels.rename(columns ={"YEAR":"year", "VARIABLE TITLE":"var_title"},inplace = True)
  labels = labels.apply(get_year_hist, axis = 1 )
  labels = labels.apply(get_week_hist, axis = 1 )
  labels = labels.apply(get_job_hist, axis = 1 )
  labels = labels.apply(get_gap_hist, axis = 1 )
  labels = labels.apply(get_gap_type_hist, axis = 1 )
  labels = labels.apply(get_Start_Stop, axis = 1)
  labels["var_name"] = labels.apply(rename_collect,args = [name], axis = 1 )

  
  #######Changing the name of the variables

  #Manually appending the four identifiers because the label files do not have these variables and 
  #they are needed while merging
  labels2 = pd.DataFrame({"RNUM":["R0000100", "R0173600", "R0214700","R0214800"], 
                      "var_name":["ID", "SAMPLE_ID", "SAMPLE_RACE","SAMPLE_SEX"]})
  labels = labels.append(labels2, ignore_index = True)

  #The actual renaming
  #df= df.rename(columns=lambda x: labels["var_name"][labels.index[labels['RNUM']==x].tolist()[0]])
  df= df.rename(columns=lambda x: lambda_rename(x, labels))
  return df 

def get_CPS_year_name(RNUM):

  CPS_year_hours_worked= [1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993]
  CPS_RNUM_hours_worked = ["R0264300","R0446900","R0702600","R0945700","R1256100","R1650900","R1923500","R2318300","R2526100","R2925100","R3127900","R3523600","R3728600","R4182600"]
  
  CPS_year_hourly_wage = [1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994]
  CPS_RNUM_hourly_wage = ["R0047010","R0263710","R0446810","R0702510","R0945610","R1256010","R1650810","R1923410","R2318210","R2526010","R2925010","R3127800","R3523500","R3728500","R4416800","R5079800"]

  if RNUM in CPS_RNUM_hourly_wage:
    label = "CPShry_wage_year" + str(CPS_year_hourly_wage[CPS_RNUM_hourly_wage.index(RNUM)]) +"_job0"
  elif RNUM in CPS_RNUM_hours_worked:
    label = "CPShrs_work_year" + str(CPS_year_hours_worked[CPS_RNUM_hours_worked.index(RNUM)]) +"_job0"
  else: 
    label = "CPS_undefined_job0"
  
  #print (label + " -> " +RNUM)
  return label


def lambda_rename(x, labels_df):
  if x in labels_df["RNUM"].tolist():
    output = labels_df["var_name"][labels_df.index[labels_df['RNUM']==x].tolist()[0]]
    if len(output) == 0:              #For hours worked CPS
      output = get_CPS_year_name(x)
  else:
    #"HOURLY RATE OF PAY CURRENT/MOST RECENT JOB" removed from hourly wages_labels but not from hourly wages. 
    output = get_CPS_year_name(x)
  return output


##########Testing######
# ####Loading the test file 
# var_name_path_test = path_weekly_array +'Var_name_test.csv'
# print(var_name_path_test)
# var_name_test = pd.read_csv(var_name_path_test)
# print(var_name_test)

# ####Running the rename code 
# var_name1 = rename_emp_hist(var_name_test,"test")
# var_name1[["Index", "var_title","got_year","got_week", "got_job", "got_gap", "var_name"]]

#Threshold Remove

Main output(s):

In [ ]:
#sub_emp_status_thresh

Only run the commented code blocks after this if this is the first time. Otherwise can load this pre saved df 

In [ ]:
#The commented code blocks after this, only run if first time. Otherwise can load this pre saved df 

sub_emp_status_thresh_path = '/content/drive/MyDrive/Alpha_beta_gamma/Coding/Data Prep/Bikalpa/Temp_df/sub_emp_status_thresh.csv'
sub_emp_status_thresh = pd.read_csv(sub_emp_status_thresh_path)
sub_emp_status_thresh.drop('Unnamed: 0', inplace=True, axis=1)


sub_emp_status_path = '/content/drive/MyDrive/Alpha_beta_gamma/Coding/Data Prep/Bikalpa/Temp_df/sub_emp_status.csv'
sub_emp_status = pd.read_csv(sub_emp_status_path)
sub_emp_status.drop('Unnamed: 0', inplace=True, axis=1)

In [ ]:
# #Loading Employment Status variables
# path_emp_status = path_weekly_array + "Employment_status.csv"
# path_emp_status_label = path_weekly_array + "Employment_status_labels.csv"
# emp_status = pd.read_csv(path_emp_status)
# emp_status_label = pd.read_csv(path_emp_status_label)

In [ ]:
# #Renaming the Employment Status variables
# emp_status_title = "emp_status"
# emp_status_renamed = rename_emp_hist(emp_status, emp_status_label,emp_status_title)

# #Some errors in NLYS
# emp_status_renamed = emp_status_renamed.rename(columns={"emp_status_year2014_week1812": "emp_status_year2014_week1912"})


In [ ]:
# def threshold_remove_identify(df, threshold, id_list):
#   df = df.sort_index(axis=1)
#  # df = df.applymap(lambda x: -1 if x ==-11 else x)
#   df["counter"] = 1
#   col_list = list(df.columns)
#   for i in range((2184-threshold)+1+1):  #1(thresh)+ ___1769___ 1(thresh) + 1, #  (2184-2*threshold)+2+1
#     sub_col_list = col_list[i : i+threshold]  #############Fix index
#     df["sum"] = df[sub_col_list].sum(axis=1)
#     df.loc[df["sum"] == -threshold, 'counter'] = 0
#   id_list.append("counter")
#   filtered = df[id_list][df.counter == 1]
#   output = filtered
#   return output

# def threshold_remove(df, threshold, id_list):
#   df1 = df.applymap(lambda x: -1 if x <=0 and x!=-99 else x)
#   identified = threshold_remove_identify(df1, threshold, id_list)
#   if "counter" in id_list: id_list.remove("counter")
#   Filtered_emp_status = (pd.merge(identified,df, on= id_list,how='inner')).drop(columns=["counter"])
#   output = Filtered_emp_status
#   return output

In [ ]:
# id_list = ["ID",	"SAMPLE_ID",	"SAMPLE_RACE",	"SAMPLE_SEX"]
# sub_emp_status = threshold_remove(emp_status_renamed, 208, id_list)
# sub_emp_status_thresh = sub_emp_status[id_list] 

In [ ]:
# with open(sub_emp_status_path, 'w', encoding = 'utf-8-sig') as f:
#   sub_emp_status.to_csv(f)

# with open(sub_emp_status_thresh_path, 'w', encoding = 'utf-8-sig') as f:
#   sub_emp_status_thresh.to_csv(f)

In [ ]:
# #Cleaning
# del [[emp_status, emp_status_label, emp_status_renamed]]
# gc.collect()
# emp_status = pd.DataFrame()
# emp_status_label = pd.DataFrame()
# emp_status_renamed = pd.DataFrame()

# Creating unique job tracker

Main output:

In [ ]:
#Unique_job_num2

Only run the commented code blocks after this if this is the first time. Otherwise can load this pre saved df 

In [ ]:
#The next two code blocks after this, only run if first time. Otherwise can load this pre saved df 
Unique_job_path = '/content/drive/MyDrive/Alpha_beta_gamma/Coding/Data Prep/Bikalpa/Temp_df/Unique_job_num2.csv'
# Unique_job_num2 = pd.read_csv(Unique_job_path)
# Unique_job_num2.drop('Unnamed: 0', inplace=True, axis=1)

In [ ]:
# path_Previous_job_num = path_summary_array + "Previous_job_num.csv"
# path_Previous_job_num_label = path_summary_array + "Previous_job_num_labels.csv"
# Previous_job_num = pd.read_csv(path_Previous_job_num)
# Previous_job_num_label = pd.read_csv(path_Previous_job_num_label)

In [ ]:
# #Rename
# Previous_job_num_title = "Previous_job_num"
# Previous_job_num_renamed = rename_emp_hist(Previous_job_num, Previous_job_num_label, Previous_job_num_title)
# #Threshold remove
# sub_Previous_job_num_renamed = pd.merge(Previous_job_num_renamed, sub_emp_status_thresh ,on= ["ID", "SAMPLE_ID", "SAMPLE_RACE","SAMPLE_SEX"],how='inner')

In [ ]:
# #Assigning a unique number to each job 
# def create_tracker_shell(df):
#   Unique_job_num = df.copy()
#   for i in range(1,6):
#     new_id = "Unique_tracker_year1979_job" + str(i)
#     Unique_job_num[new_id] = i

#   tracker = 6

#   list = Unique_job_num.columns
#   id_list = ['ID', 'SAMPLE_ID', 'SAMPLE_RACE', 'SAMPLE_SEX']
#   already_renamed_list = ['Unique_tracker_year1979_job1', 'Unique_tracker_year1979_job2', 'Unique_tracker_year1979_job3', 'Unique_tracker_year1979_job4', 'Unique_tracker_year1979_job5']
#   renaming_col = [x for x in list if (x not in id_list) and (x not in already_renamed_list)]

#   for col_name in renaming_col:
#       Unique_job_num[col_name] = tracker
#       new_name = "Unique_tracker_" + col_name[-13:]
#       Unique_job_num=Unique_job_num.rename(columns={col_name: new_name})
#       tracker = tracker + 1
#   return Unique_job_num

# Unique_job_num = create_tracker_shell(sub_Previous_job_num_renamed)

In [ ]:
# def create_unique_tracker(x, col_list, col_list_plus, prev):
#   x_prev = prev.loc[prev["ID"] == x.ID]

#   for col in col_list:
#     c_year = col[-9:-5] #current year 
#     c_job = col[-1:]   #current job 

#     prev_year = "Previous_job_num_year"+ c_year +"_job"+ c_job
#     job = int(x_prev[prev_year])

#     #There are some inconsistencies. So, this is done to ensure two jobs don't point to the same prev job
#     job_list_temp = ["1","2","3","4"]
#     if c_job in job_list_temp: job_list_temp.remove(c_job)
#     job_list=[]
#     for i in job_list_temp:
#       prev_year_j = "Previous_job_num_year"+ c_year +"_job"+i
#       job_list.append(int(x_prev[prev_year_j]))

#     if job not in job_list[:int(c_job)-1]:   
#       if job >0 and job <5:                 #Computational reasons, only job 1 to 4 considered
#         p_year = str(int(c_year)-1)
#         p_num = str(job)
#         p_var_name = "Unique_tracker_year"+p_year+"_job"+p_num
#         if p_var_name not in col_list_plus:
#           p_year = str(int(c_year)-2)
#           p_var_name = "Unique_tracker_year"+p_year+"_job"+p_num
        
#         p_tracker = x.loc[p_var_name]   #job number in that previous year as listed in unique_tracker..
#         x[col] = p_tracker
#   return x 

# id_list = ['ID', 'SAMPLE_ID', 'SAMPLE_RACE', 'SAMPLE_SEX']
# already_renamed_list = ['Unique_tracker_year1979_job1', 'Unique_tracker_year1979_job2', 'Unique_tracker_year1979_job3', 'Unique_tracker_year1979_job4', 'Unique_tracker_year1979_job5']
# tracker_col = [x for x in list(Unique_job_num.columns) if (x not in id_list) and (x not in already_renamed_list)]
# tracker_col_plus = [x for x in list(Unique_job_num.columns) if (x not in id_list)]

# Unique_job_num2 = Unique_job_num.apply(create_unique_tracker, args = [tracker_col, tracker_col_plus, sub_Previous_job_num_renamed], axis = 1 )

# #Testing 
# # previous_job_mini = sub_Previous_job_num_renamed.head(20)
# # previous_job_mini.iloc[4,5]=1
# # Unique_job_num_mini = Unique_job_num.head(20)
# # Unique_job_num2_mini = Unique_job_num_mini.apply(create_unique_tracker, args = [tracker_col, tracker_col_plus, previous_job_mini], axis = 1 )

# with open(Unique_job_path, 'w', encoding = 'utf-8-sig') as f:
#   Unique_job_num2.to_csv(f)


In [ ]:
# #Cleaning
# del [[ Previous_job_num_label, sub_Previous_job_num_renamed, Previous_job_num , Unique_job_num]] 
# gc.collect()
# Previous_job_num = pd.DataFrame()
# Previous_job_num_label = pd.DataFrame()
# sub_Previous_job_num_renamed = pd.DataFrame()
# Unique_job_num = pd.DataFrame()

# Weekly Employment History

Main output(s):

In [ ]:
#emp_hist_main    , with orginial job number 
#emp_hist_main3   , with tracker number 


Only run the commented code blocks after this if this is the first time. Otherwise can load this pre saved df 

In [ ]:
##The commented code blocks after this, only run if first time. Otherwise can load this pre saved df 
emp_hist_main3_path = '/content/drive/MyDrive/Alpha_beta_gamma/Coding/Data Prep/Bikalpa/Temp_df/emp_hist_main3.csv'
emp_hist_main3 = pd.read_csv(emp_hist_main3_path)
emp_hist_main3.drop('Unnamed: 0', inplace=True, axis=1)


emp_hist_main_path = '/content/drive/MyDrive/Alpha_beta_gamma/Coding/Data Prep/Bikalpa/Temp_df/emp_hist_main.csv'
# emp_hist_main = pd.read_csv(emp_hist_main_path)
# emp_hist_main.drop('Unnamed: 0', inplace=True, axis=1)


sub_dual_jobs_fixed_path = '/content/drive/MyDrive/Alpha_beta_gamma/Coding/Data Prep/Bikalpa/Temp_df/sub_dual_jobs_fixed.csv'
# sub_dual_jobs_fixed = pd.read_csv(sub_dual_jobs_fixed_path)
# sub_dual_jobs_fixed.drop('Unnamed: 0', inplace=True, axis=1)
# sub_dual_jobs_fixed.drop('Unnamed: 0.1', inplace=True, axis=1)

In [ ]:
# #Loading Dual Jobs variables
# path_dual_jobs = path_weekly_array + "Dual_jobs.csv"
# path_dual_jobs_label = path_weekly_array + "Dual_jobs_labels.csv"
# dual_jobs  = pd.read_csv(path_dual_jobs )
# dual_jobs_label = pd.read_csv(path_dual_jobs_label)

In [ ]:
# #Fixing some errors in the column name that stem from the NLYS
# if dual_jobs_label['RNUM'][5866]== "W1382000" and dual_jobs_label['RNUM'][5960]== "W1391400": 
#   dual_jobs_label.iloc[5866,3] = "JOB NUMBER 1 (2016) WEEK 2030"
#   dual_jobs_label.iloc[5960,3] = "JOB NUMBER 2 (2016) WEEK 2030"
# else: 
#   sys.exit("Fix this NLYS error")

In [ ]:
# #Renaming the Dual variables
# dual_jobs_title = "dual_jobs"
# dual_jobs_renamed = rename_emp_hist(dual_jobs , dual_jobs_label,dual_jobs_title)

# #Threshold remove
# sub_dual_jobs_renamed = pd.merge(dual_jobs_renamed, sub_emp_status_thresh ,on= ["ID", "SAMPLE_ID", "SAMPLE_RACE","SAMPLE_SEX"],how='inner')

In [ ]:
# #Fixing some errors in the column name that stem from the NLYS
# missing_dua11_week = ["dual_jobs_year2019_week"+str(week)+"_job1" for week in [2182, 2183, 2184]]  
# for col in missing_dua11_week: sub_dual_jobs_renamed[col] = -99

# missing_dual2_week = [ "dual_jobs_year2019_week"+str(week)+"_job2" for week in range(2168,2184+1)]  
# for col in missing_dual2_week: sub_dual_jobs_renamed[col] = -99

# dual_job3_holes_week = [0,1,2,3,4,5,6,53,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,106,107,110,111,124,133,135,137,138,139,140,161,185,186,187,189,190,191,192,193,228,229,230,232,233,234,235,236,237,238,239,240,241,242,243,244,260,261,262,264,389,390,391,393,394,395,396,398,399,400,401,417,418,419,495,496,499,500,501,531,532,533,534,535,536,537,538,546,547,548,554,555,583,584,585,586,587,588,589,590,591,592,593,594,597,2051,2052,2053,2054,2055,2056,2057,2058,2059,2060,2061,2062,2063,2064,2065,2066,2067,2068,2069,2070,2071,2072,2073,2074,2075,2076,2077,2078,2079,2080,2081,2082,2138,2139,2140,2141,2142,2143,2144,2145,2146,2147,2148,2149,2150,2151,2152,2153,2154,2155,2156,2157,2158,2159,2160,2161,2162,2163,2164,2165,2166,2167,2168,2169,2170,2171,2172,2173,2174,2175,2176,2177,2178,2179,2180,2181,2182, 2183, 2184]
# dual_job3_holes_year = [1978,1978,1978,1978,1978,1978,1978,1978,1979,1979,1979,1979,1979,1979,1979,1979,1979,1979,1979,1979,1979,1979,1979,1979,1979,1979,1979,1979,1979,1979,1979,1979,1979,1979,1979,1979,1979,1979,1979,1979,1979,1979,1979,1979,1979,1979,1979,1980,1980,1980,1980,1980,1980,1980,1980,1980,1980,1980,1981,1981,1981,1981,1981,1981,1981,1981,1981,1982,1982,1982,1982,1982,1982,1982,1982,1982,1982,1982,1982,1982,1982,1982,1982,1982,1982,1983,1983,1985,1985,1985,1985,1985,1985,1985,1985,1985,1985,1985,1985,1985,1986,1987,1987,1987,1987,1987,1988,1988,1988,1988,1988,1988,1988,1988,1988,1988,1988,1988,1988,1989,1989,1989,1989,1989,1989,1989,1989,1989,1989,1989,1989,1989,2017,2017,2017,2017,2017,2017,2017,2017,2017,2017,2017,2017,2017,2017,2017,2017,2017,2017,2017,2017,2017,2017,2017,2017,2017,2017,2017,2017,2017,2017,2017,2017,2018,2018,2018,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019]
# if (len(dual_job3_holes_week)==len(dual_job3_holes_year)):

#   missing_dua13_week = ["dual_jobs_year"+str(dual_job3_holes_year[i])+"_week"+str(dual_job3_holes_week[i])+"_job3" for i in range(0,len(dual_job3_holes_week))]  
#   minus_99 = [-99 for i in range(0,len(missing_dua13_week))] 
#   minus_99_collection = [minus_99 for i in range(0,len(sub_dual_jobs_renamed)) ]
#   missing_dua13_week_collection = pd.DataFrame(minus_99_collection, columns=missing_dua13_week)
#   sub_dual_jobs_fixed=pd.concat([sub_dual_jobs_renamed,missing_dua13_week_collection],axis=1)
# else:
#   sys.exit("The two list are not of equal length")

In [ ]:
# with open(sub_dual_jobs_fixed_path, 'w', encoding = 'utf-8-sig') as f:
#    sub_dual_jobs_fixed.to_csv(f)

In [ ]:
# #Cleaning
# del [[dual_jobs, dual_jobs_label, dual_jobs_renamed, missing_dua13_week_collection, sub_dual_jobs_renamed]] 
# gc.collect()
# dual_jobs = pd.DataFrame()
# dual_jobs_label = pd.DataFrame()
# dual_jobs_renamed = pd.DataFrame()
# missing_dua13_week_collection = pd.DataFrame()
# sub_dual_jobs_renamed = pd.DataFrame()

In [ ]:
# id_list = ['ID', 'SAMPLE_ID', 'SAMPLE_RACE', 'SAMPLE_SEX']
# emp_hist_main = pd.merge(sub_dual_jobs_fixed, sub_emp_status ,on= id_list ,how='inner')

In [ ]:
# #Standardizing the name 
# def rename_correction(df):
#   col_list_combined_pre = list(df.columns)
#   col_list_combined_post = list()
#   for col in col_list_combined_pre:
#     seg = col.split("_")
    
#     if seg[0] != "emp" and seg[0] != "dual":
#       new_name = col 
#     else:
#       zero = ""
#       #print(len(seg[3]))
#       for i in range(0,8-len(seg[3])): zero=zero+"0"
#       if seg[0] == "dual": 
#        # print(seg[3])
#         new_name = "job_" + seg[2]+"_" + "week"+zero +seg[3][4:]+"_job"+str(int(seg[4][-1])+1)  
#       elif seg[0] == "emp":
#         #print(seg[3])
#         new_name = "job_" + seg[2]+"_" + "week" +zero+seg[3][4:]+"_job1"

#     col_list_combined_post.append(new_name)

#   df.columns = col_list_combined_post
#   return df

# emp_hist_main = rename_correction(emp_hist_main)

# emp_hist_main_path = '/content/drive/MyDrive/Alpha_beta_gamma/Coding/Data Prep/Bikalpa/Temp_df/emp_hist_main.csv'
# with open(emp_hist_main_path, 'w', encoding = 'utf-8-sig') as f:
#    emp_hist_main.to_csv(f) 

In [ ]:
# emp_hist_main = pd.merge(emp_hist_main, Unique_job_num2 ,on= id_list ,how='inner')

In [ ]:
# #Cleaning before the next step because it requires a lot of RAM
# del [[sub_dual_jobs_fixed, sub_emp_status, Unique_job_num2]] 
# gc.collect()
# sub_dual_jobs_fixed = pd.DataFrame()
# sub_emp_status = pd.DataFrame()
# Unique_job_num2 = pd.DataFrame()

In [ ]:
# emp_hist_test = emp_hist_main.copy()
# #emp_hist_main2 = emp_hist_test.head(2)
# emp_hist_main2 = emp_hist_test

# #Assigning tracker number to each week

# def tracker_ordered(x, id_list, job_col, id_job_col):   
  
#   #Get the right ordering of the job 
#   uniq_values = sorted(list(set(x[sorted(job_col)])))
#   job_values = sorted([y for y in uniq_values if y<999 and y >0])
#   job_values_index1 = [list(x[job_col]).index(y) for y in job_values]
#   job_values_index2 = sorted(job_values_index1)
#   job_values_index_sorted = [job_values[job_values_index1.index(y)] for y in job_values_index2 ] 

#   #Change each job number to the right ascending order
#   for i in range(0,len(job_values_index_sorted)):
#     x[job_col] = x[job_col].replace([job_values_index_sorted[i]],str(i+1))          
    
#   x[job_col] = x[job_col].astype(int)

#   return x[id_job_col]

# def assign_tracker_number(x, id_list, job_col, tracker_col, int_year):
#   for s_year in range(0,2185):                                          #loop through every week
#     sub_job_col = [job_col[4*s_year], job_col[4*s_year+1], job_col[4*s_year+2], job_col[4*s_year+3]]         
#                                                                         #list of jobs in this week
#     survey_code_j1 = x[sub_job_col[0]]                                  
#     if survey_code_j1 >=0 and survey_code_j1 <=10:                      #If job1 survey code this, do the same for all 4 jobs
#       for column in sub_job_col: x[column] = 1000+ survey_code_j1
#     else:                                                               #Else loop through every job in the week

#       for job in sub_job_col:
#         survey_code = x[job]
#         if survey_code >= 100:                                          #If it contains job info do this 
#           frac, whole = math.modf(survey_code/100)
#           if round(frac*100) <5:                                        #Sometime job is listed are more than 4, ignore in this case
#             year = int_year[int(whole)]
#             job_num = round(frac*100)
#             relevant_tracker = "Unique_tracker_year" + str(year) +"_job" +str(job_num)  #Get the relevant tracker col
#             if year < 2019 and year >1978 and job_num> 0 and job_num < 5:
#               x[job] = x[relevant_tracker]                                              #Put the unique tracker info
#             else:
#               print(job + str(survey_code))
#               sys.exit("Error")
#           else:                                                         #If it contains a >4 job number
#             x[job] = 999
#         else:                                                           #If it contains a negative value
#           x[job] = 0
    
#   #Making the tracker number ordered, for example from [1,2,4,7,9] to [1,2,3,4,5]
#   id_job_col = id_list + job_col
#   x = tracker_ordered(x, id_list, job_col, id_job_col)

#   #Only reutrns x with id_list and job_list
#   return x


# id_list = ['ID', 'SAMPLE_ID', 'SAMPLE_RACE', 'SAMPLE_SEX']
# col_list = list(emp_hist_main.columns)
# job_col = sorted([x for x in col_list if (x not in id_list) and (x[:3] == "job" )])
# tracker_col = [x for x in col_list if (x not in id_list) and (x[:14] == "Unique_tracker" )]

# int_year = [1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1996,1998,2000,2002,2004,2006,2008,2010,2012,2014,2016,2018]
# if len(col_list) == len(id_list) + len(tracker_col) +len(job_col):
#   emp_hist_main3 = emp_hist_main2.apply(assign_tracker_number, args = [id_list, job_col, tracker_col, int_year], axis = 1 )
# else: 
#   sys.exit("Error, maybe in rename correction")


In [ ]:
# emp_hist_main3_path = '/content/drive/MyDrive/Alpha_beta_gamma/Coding/Data Prep/Bikalpa/Temp_df/emp_hist_main3.csv'
# with open(emp_hist_main3_path, 'w', encoding = 'utf-8-sig') as f:
#    emp_hist_main3.to_csv(f) 

In [ ]:
# #Cleaning 
# del [[emp_hist_main, emp_hist_main2, emp_hist_test]] 
# gc.collect()
# emp_hist_main = pd.DataFrame()
# emp_hist_main2 = pd.DataFrame()
# emp_hist_test = pd.DataFrame()

# Wage Data

Main output(s):

In [ ]:
#sub_wage
#sub_CPS_Int_check_renamed 

Only run the commented code blocks after this if this is the first time. Otherwise can load this pre saved df

In [ ]:
#The next two code blocks after this, only run if first time. Otherwise can load this pre saved df 
# sub_wage_path = '/content/drive/MyDrive/Alpha_beta_gamma/Coding/Data Prep/Bikalpa/Temp_df/sub_wage.csv'
# sub_wage = pd.read_csv(sub_wage_path)
# sub_wage.drop('Unnamed: 0', inplace=True, axis=1)

# sub_CPS_Int_check_renamed_path = '/content/drive/MyDrive/Alpha_beta_gamma/Coding/Data Prep/Bikalpa/Temp_df/sub_CPS_Int_check_renamed.csv'
# sub_CPS_Int_check_renamed = pd.read_csv(sub_CPS_Int_check_renamed_path)
# sub_CPS_Int_check_renamed.drop('Unnamed: 0', inplace=True, axis=1)

In [ ]:
# #Loading Hourly Wages variables
# path_hourly_wages = path_summary_array + "Hourly_wages.csv"
# path_hourly_wages_label = path_summary_array+ "Hourly_wages_labels.csv"
# hourly_wages = pd.read_csv(path_hourly_wages)
# hourly_wages_label = pd.read_csv(path_hourly_wages_label)
# hourly_wages_label = hourly_wages_label.loc[hourly_wages_label["VARIABLE TITLE"] != "HOURLY RATE OF PAY CURRENT/MOST RECENT JOB"]


In [ ]:
# #Renaming the hourly_wage data 
# hourly_wage_title = "hourly_wage"
# hourly_wages_renamed = rename_emp_hist(hourly_wages, hourly_wages_label,hourly_wage_title)

# #Threshold remove
# sub_hourly_wages_renamed = pd.merge(hourly_wages_renamed, sub_emp_status_thresh ,on= ["ID", "SAMPLE_ID", "SAMPLE_RACE","SAMPLE_SEX"],how='inner')


In [ ]:
# #Loading CPS int check variables
# path_CPS_Int_check = path_summary_array + "CPS_Int_check_sub.csv" #Taking in ..sub because there are some repeat col in NLYS
# path_CPS_Int_check_label = path_summary_array+ "CPS_Int_check_labels.csv"
# CPS_Int_check = pd.read_csv(path_CPS_Int_check)
# CPS_Int_check_label = pd.read_csv(path_CPS_Int_check_label)

In [ ]:
# #Renaming the CPS Int Check data 
# CPS_Int_check_title = "CPS_Int_check"
# CPS_Int_check_renamed = rename_emp_hist(CPS_Int_check, CPS_Int_check_label, CPS_Int_check_title)

# #Threshold remove
# sub_CPS_Int_check_renamed = pd.merge(CPS_Int_check_renamed, sub_emp_status_thresh , on= ["ID", "SAMPLE_ID", "SAMPLE_RACE","SAMPLE_SEX"],how='inner')

In [ ]:
# sub_wage_total_info = pd.merge( sub_hourly_wages_renamed , sub_CPS_Int_check_renamed,    on= ["ID", "SAMPLE_ID", "SAMPLE_RACE","SAMPLE_SEX"] , how='inner')

In [ ]:
#print(sorted(list(sub_wage_total_info.columns)))

In [ ]:
# sub_wage_total_info['hourly_wage_year1985_job1']

In [ ]:
# #         " current/most recent job and wage info for 5 reported jobs in each interview we redefine
# # 5 wage variables (w1-w5) that are equal to current job's wage (wagec) if original entry wi is negative (possibly entry error)
# # and wagec is not missing and positive*/  "


# def CPS_assign_wage(x , id_list, hourly_wage_columns, CPS_col, CPS_int_check_col, relv_CPS_years):
#   for y in relv_CPS_years:
#     CPS_year_col = 'CPShry_wage_year' + str(y) + '_job0'
#     CPS_wage = x[CPS_year_col]

#     if CPS_wage >0 and y != 1993:
#       for j in range(1,5):
#         hourly_wage_col = 'hourly_wage_year'+str(y)+'_job'+str(j)
#         hourly_wage = int(x[hourly_wage_col])
#         CPS_int_check_yj_col = 'CPS_Int_check_year'+str(y)+'_job'+str(j)
#         CPS_int_check_yj = int(x[CPS_int_check_yj_col])

#         if hourly_wage <0 and CPS_int_check_yj == 1: 
#           x[hourly_wage_col] = CPS_wage

#     elif CPS_wage >0 and y == 1993:         #1993 CPS int check only got one job number #1
#       hourly_wage  = x['hourly_wage_year1993_job1']
#       CPS_int_check_yj = x['CPS_Int_check_year1993_job1']
#       if hourly_wage <0 and CPS_int_check_yj == 1:
#         x[hourly_wage_col] = CPS_wage

#   return x[id_list + hourly_wage_columns]
  


# id_list = ['ID', 'SAMPLE_ID', 'SAMPLE_RACE', 'SAMPLE_SEX']
# full_col_list = list(sub_wage_total_info.columns)
# CPS_col = [x for x in full_col_list if x[0:6]=="CPShry"]
# CPS_int_check_col = [x for x in full_col_list if x[0:13]=="CPS_Int_check"]
# hourly_wage_columns = [x for x in full_col_list if x[0:11]=="hourly_wage" and int(x[-1])<5]
# int_year = [1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1996,1998,2000,2002,2004,2006,2008,2010,2012,2014,2016,2018]
# relevant_CPS_year = [1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993]


# sub_wage = sub_wage_total_info.apply( CPS_assign_wage , args = [id_list, hourly_wage_columns, CPS_col, CPS_int_check_col, relevant_CPS_year], axis = 1 )

In [ ]:
# #Inflations adjustment 
# #and
# #Reassign wages above the xth percentile to the value of the xth percentile

# import cpi
# #cpi.update()       

# percentile_upper_bound = 99
# Infl_ref_year = 2010     #Inflation base year      

# #Replace negative values with zeros 
# sub_wage[hourly_wage_columns] = np.where(sub_wage[hourly_wage_columns] < 0, 0, sub_wage[hourly_wage_columns])
# sub_wage[hourly_wage_columns] = sub_wage[hourly_wage_columns].apply(lambda x : cpi.inflate(x, int(x.name[-9:-5]) , to=Infl_ref_year))

# #Fix the xth percentile issue
# all_values = sub_wage[hourly_wage_columns].values 
# all_values2 = [item for sublist in all_values for item in sublist if item >0]
# up_per_value = np.percentile(all_values2, percentile_upper_bound)
# sub_wage[hourly_wage_columns] = np.where(sub_wage[hourly_wage_columns] > up_per_value , up_per_value, sub_wage[hourly_wage_columns])
# sub_wage = sub_wage.astype(int)

In [ ]:
# #Remove spikes from wage data because they might be errors 
# def spike_remove(df, job_num, id_list):
#   df = df.applymap(lambda x: -1 if x <0 else x)
#   df = df.sort_index(axis=1)

#   job_number = str(job_num)
#   col_list = list(df.columns)
#   sub_col_list = [x for x in col_list if (x in id_list or x[-1]==job_number)]
  
#   i = 4
#   while i +3<= len(sub_col_list):
#     sub2_col_list = sub_col_list[i:i+3]
#     df["temp_value"] = df.apply(spike_change,args = [sub2_col_list], axis = 1 )
#     df.loc[:,sub2_col_list[1]] = df["temp_value"]
#     df.drop(columns=['temp_value'])
#     i +=1

#   output = df.loc[:,sub_col_list]
#   return output


# def spike_change(row, col_list):
#   first = row[col_list[0]]
#   second = row[col_list[1]]
#   third = row[col_list[2]]
#   value_list = [first, second, third]
#   if 0 not in value_list and -1 not in value_list:
#     change_0_1 =  row[col_list[1]] / row[col_list[0]]
#     change_1_2 =  row[col_list[1]] / row[col_list[2]]
#     Spike_size = 3
#     if (abs(change_0_1) < 0.3333 and abs(change_1_2) < 0.3333) or (abs(change_0_1) > 3 and abs(change_1_2) > 3):      
#       output = None 
#     else:
#       output = second
#     #More testing is needed, with artificial sample 
#   else:
#     output = second
#   return output

# def spike_remove_alljobs(df,id_list,total_job):
#   wage1 = df[id_list]
#   for i in range(1,total_job+1):
#     wages = spike_remove(df, i , id_list)
#     wage1 = pd.merge(wage1, wages ,on= ["ID", "SAMPLE_ID", "SAMPLE_RACE","SAMPLE_SEX"],how='inner')
#   return wage1

# id_list = ["ID",	"SAMPLE_ID",	"SAMPLE_RACE",	"SAMPLE_SEX"]
# sub_wage = spike_remove_alljobs(sub_wage, id_list, 4)


In [ ]:
# with open(sub_wage_path, 'w', encoding = 'utf-8-sig') as f:
#   sub_wage.to_csv(f)

# with open(sub_CPS_Int_check_renamed_path, 'w', encoding = 'utf-8-sig') as f:
#   sub_CPS_Int_check_renamed.to_csv(f)


In [ ]:
# #Cleaning
# del [[ hourly_wages, hourly_wages_label, hourly_wages_renamed , sub_hourly_wages_renamed, CPS_Int_check, CPS_Int_check_label, CPS_Int_check_renamed, sub_wage_total_info]] 
# gc.collect()
# hourly_wages = pd.DataFrame()
# hourly_wages_label = pd.DataFrame()
# hourly_wages_renamed = pd.DataFrame()
# sub_hourly_wages_renamed = pd.DataFrame()
# CPS_Int_check = pd.DataFrame()
# CPS_Int_check_label = pd.DataFrame()
# CPS_Int_check_renamed = pd.DataFrame()
# sub_wage_total_info = pd.DataFrame()

#Hours Worked Data

In [ ]:
#hours_worked_weekly

In [ ]:
#The next code blocks after this, only run if first time. Otherwise can load this pre saved df 
hours_worked_weekly_path = '/content/drive/MyDrive/Alpha_beta_gamma/Coding/Data Prep/Bikalpa/Temp_df/hours_worked_weekly.csv'
hours_worked_weekly = pd.read_csv(hours_worked_weekly_path)
hours_worked_weekly.drop('Unnamed: 0', inplace=True, axis=1)

In [ ]:
# #Loading Hours worked data
# path_hours_worked = path_summary_array + "Hours_worked.csv"
# path_hours_worked_label = path_summary_array + "Hours_worked_label.csv"
# hours_worked = pd.read_csv(path_hours_worked)
# hours_worked_label = pd.read_csv(path_hours_worked_label)

In [ ]:
# #Renaming
# hours_worked_title = "hours_worked"
# hours_worked_renamed = rename_emp_hist(hours_worked, hours_worked_label,hours_worked_title)

# #Threshold remove
# sub_hours_worked_renamed = pd.merge(hours_worked_renamed, sub_emp_status_thresh ,on= ["ID", "SAMPLE_ID", "SAMPLE_RACE","SAMPLE_SEX"],how='inner')

In [ ]:
# #Replacing negative values with zeros

# id_list = ["ID", "SAMPLE_ID", "SAMPLE_RACE","SAMPLE_SEX"]
# hours_worked_list = [x for x in sorted(list(sub_hours_worked_renamed.columns)) if x not in id_list and int(x[-1])<5]
# sub_hours_worked_renamed[hours_worked_list] = np.where(sub_hours_worked_renamed[hours_worked_list] < 0, 0, sub_hours_worked_renamed[hours_worked_list])
# sub_hours_worked_renamed = sub_hours_worked_renamed[id_list + hours_worked_list]

In [ ]:
# #Replacing 99th percentile plus values with the 99th percentile value 

# percentile_upper_bound = 99
# all_values = sub_hours_worked_renamed[hours_worked_list].values 
# all_values2 = [item for sublist in all_values for item in sublist if item >0]
# up_per_value = np.percentile(all_values2, percentile_upper_bound)
# sub_hours_worked_renamed[hours_worked_list] = np.where(sub_hours_worked_renamed[hours_worked_list] > up_per_value , up_per_value, sub_hours_worked_renamed[hours_worked_list])
# sub_hours_worked_renamed = sub_hours_worked_renamed.astype(int)

In [ ]:
# #Fixing CPS stuff

# path_CPS_hours_worked = path_summary_array + "CPS_hours_worked.csv"
# path_CPS_hours_worked_label = path_summary_array + "CPS_hours_worked_labels.csv"
# CPS_hours_worked = pd.read_csv(path_CPS_hours_worked)
# CPS_hours_worked_label = pd.read_csv(path_CPS_hours_worked_label)

# CPS_hours_worked_title = "CPS_hours_worked"
# CPS_hours_worked_renamed = rename_emp_hist(CPS_hours_worked, CPS_hours_worked_label,CPS_hours_worked_title)

In [ ]:
# # #         " current/most recent job and wage info for 5 reported jobs in each interview we redefine
# # # 5 wage variables (w1-w5) that are equal to current job's wage (wagec) if original entry wi is negative (possibly entry error)
# # # and wagec is not missing and positive*/  "


# def CPS_assign_hrs_worked(x , id_list, hours_worked_columns, CPS_col, CPS_int_check_col, relv_CPS_years):
#   for y in relv_CPS_years:
#     CPS_year_col = 'CPShrs_work_year' + str(y) + '_job0'
#     CPS_hours_worked = x[CPS_year_col]

#     if CPS_hours_worked >0 and y != 1993:
#       for j in range(1,5):
#         hours_worked_col = 'hours_worked_year'+str(y)+'_job'+str(j)
#         hours_worked = int(x[hours_worked_col])
#         CPS_int_check_yj_col = 'CPS_Int_check_year'+str(y)+'_job'+str(j)
#         CPS_int_check_yj = int(x[CPS_int_check_yj_col])

#         if hours_worked ==0 and CPS_int_check_yj == 1: 
#           x[hours_worked_col] = CPS_hours_worked

#     elif CPS_hours_worked >0 and y == 1993:         #1993 CPS int check only got one job number #1
#       hours_worked  = x['hours_worked_year1993_job1']
#       CPS_int_check_yj = x['CPS_Int_check_year1993_job1']
#       if x['hours_worked_year1993_job1'] ==0 and CPS_int_check_yj == 1:
#         x['hours_worked_year1993_job1'] = CPS_hours_worked

#   return x[id_list + hours_worked_columns]
  

# sub_hours_worked = pd.merge(CPS_hours_worked_renamed, sub_hours_worked_renamed ,on= ["ID", "SAMPLE_ID", "SAMPLE_RACE","SAMPLE_SEX"],how='inner')
# sub_hours_worked = pd.merge(sub_hours_worked,  sub_CPS_Int_check_renamed ,on= ["ID", "SAMPLE_ID", "SAMPLE_RACE","SAMPLE_SEX"],how='inner')


# id_list = ['ID', 'SAMPLE_ID', 'SAMPLE_RACE', 'SAMPLE_SEX']
# full_col_list = list(sub_hours_worked.columns)
# CPS_col = [x for x in full_col_list if x[0:6]=="CPShrs"] 
# CPS_int_check_col = [x for x in full_col_list if x[0:13]=="CPS_Int_check"]
# hours_worked_columns = [x for x in full_col_list if x[0:12]=="hours_worked" and int(x[-1])<5]
# int_year = [1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1996,1998,2000,2002,2004,2006,2008,2010,2012,2014,2016,2018]
# relevant_CPS_year = [1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993]



# sub_hours_worked = sub_hours_worked.apply( CPS_assign_hrs_worked , args = [id_list, hours_worked_columns, CPS_col, CPS_int_check_col, relevant_CPS_year], axis = 1 )



In [ ]:
# sub_hours_worked_test = sub_hours_worked.copy(deep=True)
# emp_hist_main_path = '/content/drive/MyDrive/Alpha_beta_gamma/Coding/Data Prep/Bikalpa/Temp_df/emp_hist_main.csv'
# emp_hist_main = pd.read_csv(emp_hist_main_path)
# emp_hist_main.drop('Unnamed: 0', inplace=True, axis=1)

# def to_weekly_x(x, id_list, yearly_list, weekly_list, int_year):

#   counter = 0
#   for yj in weekly_list:
#     counter = 0
#     year = yj[-18:-14]
#     job = yj[-1]
#     job_survey_num = x[yj]
#     if job_survey_num >99:
                 
      
#       frac, whole = math.modf(job_survey_num/100)
#       if round(frac*100) <5:                                       
#           year = int_year[int(whole)]
#           job_num = round(frac*100)
#           relevant_hours_worked = "hours_worked_year" + str(year) +"_job" +str(job_num)   
          
#           if year < 2019 and year >1978 and job_num> 0 and job_num < 5:
#             hours_worked = x[relevant_hours_worked]
#             if hours_worked >0:
#               x[yj] = hours_worked
#             else: 
#               counter = counter +1
#       else:                                                        
#          x[yj] = -5     #If job 5 was listed

#   if counter > 0 : print(counter)
#   id_list = ['ID', 'SAMPLE_ID', 'SAMPLE_RACE', 'SAMPLE_SEX']
#   return x[id_list + weekly_list] 



# def to_weekly(sub_hours_worked, unique_tracker_df, emp_hist_list):
#   #the name unique_tracker is reminent for previous versions of the code. Please ignore
#   new_col_name = ["hours_worked_year" + x[8:] for x in  emp_hist_list]  
#   unique_tracker_df_copy = unique_tracker_df #.copy().sample(n=1000)
#   unique_tracker_df_copy.columns.values[4:] = new_col_name
  
#   id_list = ['ID', 'SAMPLE_ID', 'SAMPLE_RACE', 'SAMPLE_SEX']
#   weekly_list = sorted(new_col_name)
#   yearly_list = sorted([x for x in list(sub_hours_worked.columns) if x not in id_list and x not in  weekly_list])
#   # print(id_list)
#   # print(weekly_list)
#   # print(yearly_list)


#   int_year = [1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1996,1998,2000,2002,2004,2006,2008,2010,2012,2014,2016,2018]
#   unique_tracker_df_copy[weekly_list] = np.where(unique_tracker_df_copy[weekly_list] <0, 0, unique_tracker_df_copy[weekly_list])
#   unique_tracker_df_copy[weekly_list] = np.where(unique_tracker_df_copy[weekly_list] <99, -10, unique_tracker_df_copy[weekly_list])
#   sub_hours_worked = pd.merge(sub_hours_worked,  unique_tracker_df_copy ,on= id_list ,how='inner')
#   sub_hours_worked_weekly = sub_hours_worked.apply(to_weekly_x , args = [id_list, yearly_list, weekly_list, int_year], axis =1) 
  
#   return sub_hours_worked_weekly 

# emp_hist_columns = list(emp_hist_main.columns)
# emp_hist_columns2 = emp_hist_columns.copy()
# id_list = ['ID', 'SAMPLE_ID', 'SAMPLE_RACE', 'SAMPLE_SEX']
# emp_hist_list = [x for x in emp_hist_columns2 if x not in id_list]
# # print(emp_hist_list)

# hours_worked_weekly = to_weekly(sub_hours_worked_test, emp_hist_main , emp_hist_list)


In [ ]:
# hours_worked_weekly_path = '/content/drive/MyDrive/Alpha_beta_gamma/Coding/Data Prep/Bikalpa/Temp_df/hours_worked_weekly.csv'
# with open(hours_worked_weekly_path, 'w', encoding = 'utf-8-sig') as f:
#   hours_worked_weekly.to_csv(f)

#Education

In [ ]:
#Loading Education data
path_HGC = path_main_edu + "Highest_grade_completed.csv"
path_HGC_label = path_summary_array + "Hours_worked_label.csv"
HGC = pd.read_csv(path_HGC)
HGC_label = pd.read_csv(path_HGC_label)

#To weekly

In [ ]:
# sub_wage_copy = sub_wage.copy(deep=True)
# emp_hist_main_path = '/content/drive/MyDrive/Alpha_beta_gamma/Coding/Data Prep/Bikalpa/Temp_df/emp_hist_main.csv'
# emp_hist_main = pd.read_csv(emp_hist_main_path)
# emp_hist_main.drop('Unnamed: 0', inplace=True, axis=1)

# def to_weekly_x(x, name, id_list, yearly_list, weekly_list, int_year):

#   counter = 0
#   for yj in weekly_list:
#     counter = 0
#     year = yj[-18:-14]
#     job = yj[-1]
#     job_survey_num = x[yj]
#     if job_survey_num >99:
                 
      
#       frac, whole = math.modf(job_survey_num/100)
#       if round(frac*100) <5:                                       
#           year = int_year[int(whole)]
#           job_num = round(frac*100)
#           relevant_col = name+"_year" + str(year) +"_job" +str(job_num)       
          
#           if year < 2019 and year >1978 and job_num> 0 and job_num < 5:
#             hours_worked_or_wage = x[relevant_col]
#             if hours_worked_or_wage  >0:
#               x[yj] = hours_worked_or_wage 
#             else: 
#               counter = counter +1
#       else:                                                        
#          x[yj] = -5     #If job 5 was listed

#   if counter > 0 : print(counter)
#   id_list = ['ID', 'SAMPLE_ID', 'SAMPLE_RACE', 'SAMPLE_SEX']
#   return x[id_list + weekly_list] 



# def to_weekly(name, sub_df, unique_tracker_df, emp_hist_list):
#   #the name unique_tracker is reminent for previous versions of the code. Please ignore
#   new_col_name = [name +"_year"+ x[8:] for x in  emp_hist_list]  
#   unique_tracker_df_copy = unique_tracker_df #.copy().sample(n=10)
#   unique_tracker_df_copy.columns.values[4:] = new_col_name
  
#   id_list = ['ID', 'SAMPLE_ID', 'SAMPLE_RACE', 'SAMPLE_SEX']
#   weekly_list = sorted(new_col_name)
#   yearly_list = sorted([x for x in list(sub_df.columns) if x not in id_list and x not in  weekly_list])
#   # print(id_list)
#   # print(weekly_list)
#   # print(yearly_list)


#   int_year = [1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1996,1998,2000,2002,2004,2006,2008,2010,2012,2014,2016,2018]
#   unique_tracker_df_copy[weekly_list] = np.where(unique_tracker_df_copy[weekly_list] <0, 0, unique_tracker_df_copy[weekly_list])
#   unique_tracker_df_copy[weekly_list] = np.where(unique_tracker_df_copy[weekly_list] <99, -10, unique_tracker_df_copy[weekly_list])
#   sub_df = pd.merge(sub_df,  unique_tracker_df_copy ,on= id_list ,how='inner')
#   sub_df_weekly = sub_df.apply(to_weekly_x , args = [name, id_list, yearly_list, weekly_list, int_year], axis =1) 
  
#   return sub_df_weekly 

# #This part could be inside
# emp_hist_columns = list(emp_hist_main.columns)
# emp_hist_columns2 = emp_hist_columns.copy()
# id_list = ['ID', 'SAMPLE_ID', 'SAMPLE_RACE', 'SAMPLE_SEX']
# emp_hist_list = [x for x in emp_hist_columns2 if x not in id_list]
# # print(emp_hist_list)


# #Fix the name here and above
# hourly_wage_weekly = to_weekly("hourly_wage",sub_wage_copy, emp_hist_main , emp_hist_list)
# hourly_wage_weekly


In [ ]:
hourly_wage_weekly_path = '/content/drive/MyDrive/Alpha_beta_gamma/Coding/Data Prep/Bikalpa/Temp_df/hourly_wage_weekly.csv'
# with open(hourly_wage_weekly_path, 'w', encoding = 'utf-8-sig') as f:
#   hourly_wage_weekly.to_csv(f)

hourly_wage_weekly = pd.read_csv(hourly_wage_weekly_path)
hourly_wage_weekly.drop('Unnamed: 0', inplace=True, axis=1)

In [ ]:
#hourly_wage_weekly.where(hourly_wage_weekly >0).count(axis=1)

#Joining

In [ ]:
hours_worked_weekly

In [ ]:
hourly_wage_weekly

In [ ]:
emp_hist_main3

In [ ]:
emp_hist_final = pd.merge(hours_worked_weekly, emp_hist_main3 ,on= ["ID", "SAMPLE_ID", "SAMPLE_RACE","SAMPLE_SEX"],how='inner')
emp_hist_final = pd.merge(emp_hist_final, hourly_wage_weekly ,on= ["ID", "SAMPLE_ID", "SAMPLE_RACE","SAMPLE_SEX"],how='inner')

In [ ]:
#Rename the columns to be more wide_to_long friendly
#do wide to long 
#Generate year
#Then K-means after possible some cross validation 
#Then education

In [ ]:
#Cleaning
# del [[ hours_worked_weekly, hourly_wage_weekly, emp_hist_main3]] 
# gc.collect()
# hours_worked_weekly = pd.DataFrame()
# hourly_wage_weekly = pd.DataFrame()
# emp_hist_main3 = pd.DataFrame()


In [ ]:
col_list = list(emp_hist_final.columns)
id_list = ['ID', 'SAMPLE_ID', 'SAMPLE_RACE', 'SAMPLE_SEX']
renaming_list = [x for x in col_list if x not in id_list]
new_name_list = id_list
a = 0
b=0
c=0

for x in renaming_list:
  if x[:11] == "hourly_wage": new_name_list.append("hourly_wage_job"+ x[-1]+"-"+x[25:29])
  if x[:3] == "job": new_name_list.append("job_tracker_job"+ x[-1]+"-"+x[17:21])
  if x[:12] == "hours_worked": new_name_list.append("hours_worked_job"+ x[-1]+"-"+x[26:30])

print(new_name_list)
emp_hist_final.columns = new_name_list
print(len(list(emp_hist_final.columns)) == len(set(list(emp_hist_final.columns))))
emp_hist_final

In [ ]:
s_name_1 = ['hourly_wage_job1','hourly_wage_job2','hourly_wage_job3','hourly_wage_job4']
s_name_2 = ['job_tracker_job1','job_tracker_job2','job_tracker_job3','job_tracker_job4']
s_name_3 = ['hours_worked_job1','hours_worked_job2','hours_worked_job3','hours_worked_job4']
s_name = s_name_1 + s_name_2 + s_name_3
     
reshaped = pd.wide_to_long(emp_hist_final, stubnames = s_name, i=id_list, j='week', sep='-')

In [ ]:
reshaped_temp_path = '/content/drive/MyDrive/Alpha_beta_gamma/Coding/Data Prep/Bikalpa/Temp_df/reshaped_temp.csv'
with open(reshaped_temp_path, 'w', encoding = 'utf-8-sig') as f:
  reshaped.to_csv(f)
   
# reshaped_temp = pd.read_csv(reshaped_temp_path)
# reshaped_temp.drop('Unnamed: 0', inplace=True, axis=1)

Appendix

In [ ]:
# sub_hours_worked_test = sub_hours_worked.copy(deep=True)
# emp_hist_main3 = pd.read_csv(emp_hist_main3_path)
# emp_hist_main3.drop('Unnamed: 0', inplace=True, axis=1)

# def to_weekly_x(x, id_list, yearly_list, weekly_list):
#   print("..........")
#   for yj in yearly_list:
#     year = yj[-9:-5]
#     job = yj[-1]
#     if x[yj] >0 or x[yj]<=0:
#       rel_col = [y for y in weekly_list if  y[-18:-14] == year  and y[-1]== job]  
#       tracker_list = []
#       for col in rel_col:
#         tracker = x[col]
#         if tracker!=0 and tracker < 9999 and tracker not in tracker_list: tracker_list.append(tracker)
#       if len(tracker_list) >0:
#         print(tracker_list )
#         print(year + "_"+job)
#         print("_")
    
#   return x 


# def to_weekly(sub_hours_worked, unique_tracker_df, unique_tracker_list):

#   new_col_name = ["hours_worked_year" + x[8:] for x in  unique_tracker_list]  
#   unique_tracker_df_copy = unique_tracker_df.copy().sample(n=2)
#   unique_tracker_df_copy.columns.values[4:] = new_col_name
  
#   id_list = ['ID', 'SAMPLE_ID', 'SAMPLE_RACE', 'SAMPLE_SEX']
#   weekly_list = new_col_name
#   yearly_list = [x for x in list(sub_hours_worked.columns) if x not in id_list and x not in  weekly_list]
#   print(id_list)
#   print(weekly_list)
#   print(yearly_list)

#   unique_tracker_df_copy[weekly_list] = np.where(unique_tracker_df_copy[weekly_list] >= 999, 9999, -unique_tracker_df_copy[weekly_list])
#   sub_hours_worked = pd.merge(sub_hours_worked,  unique_tracker_df_copy ,on= id_list ,how='inner')
#   sub_hours_worked_weekly = sub_hours_worked.apply(to_weekly_x , args = [id_list, yearly_list, weekly_list], axis =1) 
  
#  #return sub_hours_worked #sub_hours_worked_weekly

# emp_hist_columns = list(emp_hist_main3.columns)
# emp_hist_columns2 = emp_hist_columns.copy()
# id_list = ['ID', 'SAMPLE_ID', 'SAMPLE_RACE', 'SAMPLE_SEX']
# unique_tracker_list = [x for x in emp_hist_columns2 if x not in id_list]


# to_weekly(sub_hours_worked_test, emp_hist_main3 , unique_tracker_list)


Summary Stat

In [ ]:
# emp_hist_final = emp_hist_final_temp.replace(0,np.nan)    
# emp_hist_final

In [ ]:
# job_2 = [x for x in list(emp_hist_final_temp.columns) if x not in id_list and (str(x[-1])=="2" and (x in list(sub_wage.columns) or (x in list(sub_hours_worked_renamed)) ) )]
# job_2_summary = emp_hist_final_temp[job_2].describe().astype(int)

# job_2_path = '/content/drive/MyDrive/Alpha_beta_gamma/Coding/Data Prep/Bikalpa/Temp_df/job_2_summary.csv'
# with open(job_2_path, 'w', encoding = 'utf-8-sig') as f:
#   job_2_summary.to_csv(f)

In [ ]:
# job_1 = [x for x in list(emp_hist_final_temp.columns) if x not in id_list and (str(x[-1])=="1" and (x in list(sub_wage.columns) or (x in list(sub_hours_worked_renamed)) ) )]
# job_1_summary = emp_hist_final_temp[job_1].describe().astype(int)

# job_1_path = '/content/drive/MyDrive/Alpha_beta_gamma/Coding/Data Prep/Bikalpa/Temp_df/job_1_summary.csv'
# with open(job_1_path, 'w', encoding = 'utf-8-sig') as f:
#   job_1_summary.to_csv(f)